In [ ]:
import tensorflow as tf
import keras
import numpy as np
from sklearn import preprocessing

import random, sys, os, argparse, json

## The Neuron

<img src="http://cs231n.github.io/assets/nn1/neuron_model.jpeg" alt="neuron" width=400/>

The figure above shows the most fundamental unit of a neural network, the __neuron__. This unit is essentially what gives neural networks the ability to learn complex datasets. The input to the unit is a vector $\vec{x} = \{x_1, x_2, ..., x_n\}$. These values are like the features (dimensions) of a dataset, which we have been using since the first notebook. The unit itself consists of a vector $\vec{w} = \{w_1, w_2, ..., w_3\}$ whose values are the __weights__, with an additional value $b$ which is the __bias__. Finally, the output of the unit is the weighted sum of the input (including the bias), followed by an __activation function__. Thus, the mathematical definition of the neuron is as follows:

$$a = \sigma(\vec{w} \cdot \vec{x} + b)$$

__Question__: This model is actually identical to logistic regression; can you identify what model you get if you remove the activation function? _Hint: it's a model that we studied in the "Supervised Learning" notebook._

### Activation Functions: The Secret Sauce of Success

The key ingredient of the perceptron which makes it so useful is the activation function. This function is typically a nonlinear function whose output range is something small such as $[0, 1]$. One of the earliest examples of an activation function is the __sigmoid__ function:

$$\sigma(x) = \frac{1}{1 + e^{-x}}$$

Here's an interactive plot to give you some intuition for how the sigmoid behaves:

The activation function has two purposes in a neuron:

1. Apply a nonlinear transformation to the weighted sum.
2. "Squash" the weighted sum, which may be large, into a small range of possible values.

The __nonlinearity__ property is especially crucial because __it allows the neuron to model nonlinear relationships__, and, as we will see later, it allows us to scale our neural network to be as large as we need it to be for a given task. Interestingly enough, the neuron model can work with all sorts of nonlinear functions, although some functions work better than others.

The __squashing__ (or __saturation__) property actually comes from the neuron's original conception as a model of a biological neuron. The idea is that a biological neuron essentially behaves like a switch with a smooth transition in the middle, so even if the weighted sum of the inputs is extremely high or extremely low, the final output of the neuron will still be a simple "on" or "off". That being said, not all activation functions adhere to this squashing property as strictly as sigmoid, so this property is really more of a guideline rather than a necessity.

## The Multilayer Perceptron

The neuron is the fundamental building block of a neural network. The process of constructing a neural network out of neurons is fairly straightforward: we first construct a __layer__ as an array of neurons, and then we construct a __network__ as a sequence of layers. This architecture is described in the figure below:

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="neural-net" width=500/>

This type of neural network is called a __multilayer perceptron (MLP)__ (or __feed-forward neural network__). The white circles in the figure are invididual neurons (we'll call them __units__ from here on out), and the colored rectangles are layers. There are three types of layers in the figure: __input layers__, __output layers__ and __hidden layers__. Hidden layers and output layers are really the same, the output layer is just the last layer in the network. The input layer is really just a placeholder for the input features (i.e. the white circles in the input layer are just input features, not neurons). We would call the network in this figure a "3-layer neural network" (again, we don't count the input layer since it's just a placeholder).

The defining aspect of an MLP is how each layer is connected: each unit in a layer receives an input from every unit in the previous layer. A layer with this kind of input connections is called a __fully-connected layer__ (or __dense layer__). Remember that each input to a unit has a weight; just as we represented the set of weights for a single unit as a vector, we can represent all of the input connections to a layer as a __weight matrix__ $W$. This matrix has size $n \times m$, where $m$ is the number of units in the previous layer and $n$ is the number of units in the given layer, and the $i^{th}$ row is just the weight vector for the $i^{th}$ unit in the given layer. Using this weight matrix we can actually compute the output of every unit in a layer with a single equation:

$$\vec{a} = \sigma(W \vec{x} + \vec{b})$$

Where $\vec{a}$ is a vector of the unit outputs, and we assume that all of the units use the same activation function. Since we're using a lot of vectors and matrices now, here are their sizes:

$$\vec{a}, \vec{b} \in \mathbb{R}^{n}; W \in \mathbb{R}^{n \times m}; \vec{x} \in \mathbb{R}^{m}$$

Now that we can construct a layer, compute the output of a layer, and connect layers in sequence, the last step is to compute the output of an entire network from input layer to output layer. This step is actually very simple: the output $\vec{a}$ of one layer becomes the input $\vec{x}$ to the next layer. For example, if we consider the 3-layer network from the figure, we can compute the output $\vec{a}_{out}$ of the network as follows:

$$f_i(\vec{x}) = \sigma(W_i \vec{x} + \vec{b}_i)$$

$$\vec{a}_{out} = f_{net}(\vec{x}) = f_3(f_2(f_1(\vec{x})))$$

This computation is also called the __forward pass__ of a network.

As you can imagine, there really is no limit to the possible size of the network -- you can have as many fully-connected layers as you want, you can have as many units per layer as you want, and you can size each layer indepedently. As a result, the MLP is an extremely general model, meaning that it can model all kinds of functions using a series of matrix transformations and nonlinear activations. This idea is called the __universal approximation theorem__, because the idea is that the MLP is able to _approximate_ any kind of function. Whether the MLP can learn any function _from data_... well, you just have to try it and see.

The __network architecture__ -- that is, the number and respective sizes of each layer, and the choice of activation function -- is a major hyperparameter of a neural network. In fact, we've really brought hyperparameters to a new level here, because the architecture could be comprised of an entire list of hyperparameters! This freedom is good because it makes neural networks extremely versatile, but it is also very daunting if we don't know how to pick the right settings for the task. You'll be experimenting with network architecture for your assignment, but for now let's look at some basic applications of neural networks.

## Supervised Learning: MLP

Let's build a neural network to perform classification on the MNIST dataset. As it turns out, scikit-learn actually has an `MLPClassifier`, but we need something that will run on a GPU -- trust me, we're going to need the extra compute power later on. From now on we're going to use two Python libraries, __Tensorflow__ and __Keras__. Tensorflow is a GPU-accelerated framework for creating and running neural networks, and Keras is a thin wrapper over Tensorflow which provides a much simpler interface (Keras also supports other _backends_ aside from Tensorflow). In short, we will use Keras, but just know that Tensorflow is doing all of the hard work behind the scenes. Now let's make a network:

In [ ]:
# Don't worry about this entire block. It is code we have written to help us access the data really easily!
# If you are interested, then go ahead and read it and ask questions later, but for now let's skip it.
class data_t(object):
    def __init__(self, data, labels):
        self.labels = labels
        self.data = data
        self.num_examples = data.shape[0]

    def next_batch(self, batch_size, index):
        idx = index * batch_size
        n_idx = index * batch_size + batch_size
        return self.data[idx:n_idx, :], self.labels[idx:n_idx, :]

class DataContainer:
    def __init__(self, data, total_gene_list=None, sub_gene_list=None, train_split=70, test_split=30):
        self.num_classes = len(data)
        self.label_names_ordered = []
        self.class_counts = {}
        self.train, self.test = self.split_set(data, total_gene_list, sub_gene_list, train_split, test_split)


    #
    # USAGE:
    # 		create a new data dictionary with classes as keys that contain a subsample of original data,
    #		specified by the 'sub_gene_list'
    # PARAMS:
    #		orig_data:       dictionary containing classes as keys, with values as matrix of class samples
    #						 with all possible genes
    #		total_gene_list: list of every gene in the original data GEM
    #   	sub_gene_list:   specified genes from a subset of total gene list
    #
    def extract_requested_genes(self, orig_data, total_gene_list, sub_gene_list):

        #get genes in sub_gen_list from total_gene_list
        gene_indexes = []
        for i in range(len(sub_gene_list)):
            gene_indexes.append(np.argwhere(total_gene_list == sub_gene_list[i]))

        # dictionary for requested data
        req_data = {}

        # iterate through dictionary, replace old data matrix with reduced data matrix
        for k in sorted(orig_data.keys()):
            reduced_data = np.zeros((len(gene_indexes), orig_data[k].shape[1]))

            for idx in xrange(0, len(gene_indexes)):
                reduced_data[idx] = orig_data[k][gene_indexes[idx][0],:]

            req_data[k] = reduced_data

        return req_data


    #
    # USAGE:
    #	shuffle the data and labels in the same order for a data set and transform the data and labels into numpy arrays
    # PARAMS:
    #	data:	the data values for a dataset
    # 	labels: the labels associated with data for a dataset
    #
    def shuffle_and_transform(self, data, labels):
        new_data = []
        new_labels = []

        samples = random.sample(xrange(len(data)),len(data))

        for i in samples:
            new_data.append(data[i])
            new_labels.append(labels[i])

        # convert lists to numpy arrays
        np_data = np.asarray(new_data)
        np_labels = np.asarray(new_labels)

        return data_t(np_data,np_labels)

    #
    # USAGE:
    #	TODO: What is this use @Colin
    def shuffle(self):
        idxs = np.arange(self.train.data.shape[0])
        idxs = np.random.shuffle(idxs)
        self.train.data = np.squeeze(self.train.data[idxs])
        self.train.labels = np.squeeze(self.train.labels[idxs])



    #
    # USAGE:
    #       split the data in data dictionary to the specified weights
    # PARAMS:
    #		data: 			 dictionary containing all samples of each class, with the class name being the key
    #       total_gene_list: list of every sorted gene in the GEM
    #       sub_gene_list:   list of genes in the subset wished to be extracted. this can be the total gene list
    #						 if all genes are wanting to be used
    #       train_split:     percentage (in integer form) for training class split
    #       test_split:      percentage (in integer form) for testing class split
    #
    def split_set(self, data, total_gene_list=None, sub_gene_list=None, train_split=70, test_split=30):

        if test_split + train_split != 100:
            print('Test and train split must sum to 100!')
            sys.exit(1)

        train_data = []
        train_labels = []
        test_data = []
        test_labels = []

        if sub_gene_list is not None:
            data = self.extract_requested_genes(data, total_gene_list, sub_gene_list)

        idx = 0 # keep count of index in dictionary

        # gather training and testing examples and labels into a list by randomly selecting indices
        # of the amount of data in each class
        for k in sorted(data.keys()):
            self.label_names_ordered.append(k)
            self.class_counts[k] = data[k].shape[1]

            num_train = int(data[k].shape[1] * train_split / 100)

            samples = random.sample(xrange(data[k].shape[1]),data[k].shape[1])
            samples_train = samples[0:num_train]
            samples_test = samples[num_train:]

            for i in xrange(len(samples_train)):
                train_data.append(data[k][:,samples_train[i]])

                label = np.zeros(self.num_classes)
                label[idx] = 1

                train_labels.append(label)

            for i in xrange(len(samples_test)):
                test_data.append(data[k][:,samples_test[i]])

                label = np.zeros(self.num_classes)
                label[idx] = 1

                test_labels.append(label)

            idx = idx + 1

        train = self.shuffle_and_transform(train_data, train_labels)
        test = self.shuffle_and_transform(test_data, test_labels)

        return [train, test]

In [ ]:
# this function will help us get the data into a format we can work with and load into the DataContainer
def load_data(num_samples_json, gtex_gct_flt):
    sample_count_dict = {}
    with open(num_samples_json) as f:
        sample_count_dict = json.load(f)

    idx = 0
    data = {}

    for k in sorted(sample_count_dict.keys()):
        data[k] = gtex_gct_flt[:,idx:(idx + int(sample_count_dict[k]))]
        idx = idx + int(sample_count_dict[k])

    return data

In [ ]:
# we are going to load in the gtex GEM from the /zfs directory on palmetto
# we also need the list of genes and a JSON that me and csheare created a while pack
# that documents the number of samples in each class
gtex_gct_flt = np.load('/zfs/feltus/ctargon/gems/gtex_gct_data_float_v7.npy')
total_gene_list = np.load('/zfs/feltus/ctargon/gene_lists/gtex_gene_list_v7.npy')
data = load_data('/zfs/feltus/ctargon/class_counts/gtex_tissue_count_v7.json', gtex_gct_flt)

In [ ]:
# we are now going to load the data into the DataContainer object so we can easily access it...
dataset = DataContainer(data, total_gene_list)

In [ ]:
# let's define some metavariables we might use later...
num_classes = len(data) # data variable is a dictionary with each key being a class
num_genes = dataset.train.data.shape[-1] # train.data is num_samples x num_genes

In [ ]:
# now it is time to create our model, preprocess our data, and get classifying
# create a neural network with three hidden layers
mlp = keras.models.Sequential()
mlp.add(keras.layers.Dense(units=1024, activation="relu", input_shape=(num_genes,))) 
mlp.add(keras.layers.Dense(units=512, activation="relu"))
mlp.add(keras.layers.Dense(units=128, activation="relu"))
mlp.add(keras.layers.Dense(units=num_classes, activation="softmax"))

In [ ]:
# lets compile the model and check out the stats
# compile the model
mlp.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# print a summary of the model
mlp.summary()

In [ ]:
# lets do a little preprocessing, then fit the model
scaler = preprocessing.MinMaxScaler() #preprocessing.MaxAbsScaler()
dataset.train.data = scaler.fit_transform(dataset.train.data)
dataset.test.data = scaler.fit_transform(dataset.test.data)

In [ ]:
# train the model
history = mlp.fit(x=dataset.train.data, y=dataset.train.labels, batch_size=128, epochs=10, validation_split=0.1)

In [ ]:
# plot the training accuracy
plt.plot(history.history["acc"])
plt.plot(history.history["val_acc"])
plt.title("Training Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Training", "Validation"], loc="upper left")
plt.show()

# plot the training loss
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Training Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Training", "Validation"], loc="upper left")
plt.show()

In [ ]:
# evaluate the model on the test set
scores = mlp.evaluate(x=dataset.test.data, y=dataset.test.labels)

# print results
for name, score in zip(mlp.metrics_names, scores):
    print("%s: %g" % (name, score))